# Quick Debugging using `icecream`

If you've been writing Python code, then at some point you've probably used some `print()` statements to help you either debug your code or understand the code flow. Sure, we have several debugging tools at our disposal, but sometimes it's just easier to print things out, maybe inside a loop to quickly examine the values of various variables as the code is running without having to break and inspect variables.

The `icecream` library can be very useful as a more advanced, flexible replacement of `print()` while typing less code:

Library documentation can be found [here](https://github.com/gruns/icecream).

First thing is you'll need to install `icecream` in your virtual env, which can be done using:
```bash
pip install icecream
```

> Note: If you plan on running `icecream` in a Python REPL (such as a Jupyter notebook), then you will likely need to ensure that the `executing` dependency `icecream` depends on, uses the latest version. To do so you can use:
>```bash
>pip install executing>=2.0.0
>```

Next, we need to import it:

In [1]:
from icecream import ic

At it's simplest, we can just use `ic` as a replacement for `print`:

In [2]:
for i in range(5):
    ic(i)

ic| i: 0
ic| i: 1
ic| i: 2
ic| i: 3
ic| i: 4


If you've ever used `print` statements to indentify what parts of your code are running using something like this:

In [3]:
import random

min_ = 100
max_ = 200
mean = (min_ + max_) / 2
sample_size = 10
below_or_at_mean = []
above_mean = []

random.seed(0)
for i in range(sample_size):
    num = random.randint(min_, max_)
    print(f"{num=}")
    if num <= mean:
        print(1)
        below_or_at_mean.append(num)
    else:
        print(2)
        above_mean.append(num)
        

num=149
1
num=197
2
num=153
2
num=105
1
num=133
1
num=165
2
num=162
2
num=151
2
num=200
2
num=138
1


Using `icecream` you can do this instead:

In [4]:
import random

min_ = 100
max_ = 200
mean = (min_ + max_) / 2
sample_size = 10
below_or_at_mean = []
above_mean = []

random.seed(0)
for i in range(sample_size):
    num = random.randint(min_, max_)
    ic(num)
    if num <= mean:
        ic()
        below_or_at_mean.append(num)
    else:
        ic()
        above_mean.append(num)

ic| num: 149
ic| 377800592.py:15 in <module> at 14:19:21.626
ic| num: 197
ic| 377800592.py:18 in <module> at 14:19:21.635
ic| num: 153
ic| 377800592.py:18 in <module> at 14:19:21.643
ic| num: 105
ic| 377800592.py:15 in <module> at 14:19:21.653
ic| num: 133
ic| 377800592.py:15 in <module> at 14:19:21.662
ic| num: 165
ic| 377800592.py:18 in <module> at 14:19:21.670
ic| num: 162
ic| 377800592.py:18 in <module> at 14:19:21.679
ic| num: 151
ic| 377800592.py:18 in <module> at 14:19:21.688
ic| num: 200
ic| 377800592.py:18 in <module> at 14:19:21.696
ic| num: 138
ic| 377800592.py:15 in <module> at 14:19:21.705


Maybe not very useful in a Jupyter notebook, but can come in handy in a Python app, where the `ic` output will tell you the module, enclosing function (if any), and the line number in the module file - that way you can easily tell the path your code execution is taking.

The nice thing about `ic` is that, unlike `print`, not only will it provide the print output, but it also **returns** the value of the expression.

This means you can easily add these prints without having to add lines of code:

In [5]:
import random

min_ = 100
max_ = 200
mean = (min_ + max_) / 2
sample_size = 10
below_or_at_mean = []
above_mean = []

random.seed(0)
for i in range(sample_size):
    num = ic(random.randint(min_, max_))
    if num <= mean:
        below_or_at_mean.append(num)
    else:
        above_mean.append(num)

ic| random.randint(min_, max_): 149
ic| random.randint(min_, max_): 197
ic| random.randint(min_, max_): 153
ic| random.randint(min_, max_): 105
ic| random.randint(min_, max_): 133
ic| random.randint(min_, max_): 165
ic| random.randint(min_, max_): 162
ic| random.randint(min_, max_): 151
ic| random.randint(min_, max_): 200
ic| random.randint(min_, max_): 138


In [6]:
below_or_at_mean

[149, 105, 133, 138]

In [7]:
above_mean

[197, 153, 165, 162, 151, 200]

Or possibly in cases such as this:

In [8]:
def my_func(a, b):
    return a + b

sum_ = ic(my_func(10, 20))
sum_

ic| my_func(10, 20): 30


30

In fact, since `ic` returns any results, you can even inject it into your code this way:

In [9]:
def my_func(a, b):
    return ic(a) + ic(b)

In [10]:
my_func(10, 20)

ic| a: 10
ic| b: 20


30

In [11]:
ic(my_func(10, 20))

ic| a: 10
ic| b: 20
ic| my_func(10, 20): 30


30

The nice thing also about `ic` is that you can entirely enable or disable it's functionality without having to rewrite any code (unlike print statement that you then have to pull out of your final code).

In [12]:
my_func(10, 20)

ic| a: 10
ic| b: 20


30

In [13]:
ic.disable()

In [14]:
my_func(10, 20)

30

We can also reenable it:

In [15]:
ic.enable()

In [16]:
my_func(10, 20)

ic| a: 10
ic| b: 20


30

Later, I'll also show you some of the configuration options available for `ic`.

Let's implement a merge sort algorithm, and we'll use `ic` to get insight into what's happening.

Now, this video is not about merge sorts, so I won't spend a lot of time explaining how merge sorts work.

Basically, mrge sort is a recursive algorithm. 

Our initial problem is how to sort a list containing N elements.

We certainly don't know how to do that easily, but what if the list is a **single** element? Can we sort that?

Of course, the answer is **yes**, the list, if it contains a single element is already sorted.

So, the merge sort approach is to reduce sorting to just sorting a single element array.

To do that we keep splitting the array into smaller and smaller parts until we reach arrays of single elements. The trick then becomes how to reassemble these now sorted sub-arrays into a single sorted array - this is the **merge** step of the MergeSort algorithm.

Let's look at a simple example and do the work manually, step by step:

Start with a list `[4, 3]`

Since that list has size `2`, we split into two separate lists, which will end up being lists of size `1`:
```
l1 = [4]
l2 = [3]
```

We have now two **sorted** lists - we just need to merge then back (maintaining the sort order).

To do that we iterate through both lists - at each iteration we look at both lists and pick the smalles (or largest, depending on what ordering you want), and add that to our final sorted list.

Here, we start with an empty results list:

```results = []```

Next, we look at the first element of both lists: `4` and `3`. We pick `3` and that to our results (and now "discard" the element we just "took" from `l2` - in practice we don't delete anything, we just maintain an index value which tells us which element to look at in each list). So now we end up with:

```
results = [3]
l1 = [4]
l2 = []
```

We iterate again, this time `l2` is "empty", and the only value we have is from `l1`, so we pick that number (it's the smallest), and we end up with:

```results = [3, 4]
l1 = []
l2 = []
```

Since both `l1` and `l2` are "empty", we are done with the sort.

Let's do this again, but this time with `5` elements:

```
data = [2, 1, 4, 3, 5]
results = []
```

**Step 1:**
```
l1 = [2, 1, 4]
l2 = [3, 5]
```

**Step 2:**:
```
l1_1 = [2, 1]
l1_2 = [4]
```

**Step 3:**
```
l1_1_1 = [2]
l1_1_2 = [1]
```
merging -> `l1_1_sorted` is the sorted list for `l1_1` --> `[1, 2]`

**Step 4:**
`l1_1` has been sorted, and `l1_2` is already sorted, so merge!
merge `sorted_l1_1` with `l1_2`. This means we have now sorted `l1`, let's call that list `l1_sorted` --> `[1, 2, 4]`

**Step 5:**
We still have to sort `l2` before we can merge it with `l1_sorted`. 
l2_1 = [3]
l2_2 = [5]

**Step 6:**
`l2_1` is already sorted (`1` element), no more work needed here.
`l2_2` is already sorted(`1` element), no more work needed here.
Merge --> `l2_sorted` --> `[3, 5]`

**Step 7:**
We now have `sorted_l1` and `sorted_l2`, and we can now merge them to obtain our final sorted array:
```
sorted_l1 = [1, 2, 4]
sorted_l2 = [3, 5]
```

merge --> `[1, 2, 3, 4, 5]`



So now let's implement our merge sort algorithm in Python - we know we'll need a separate function for merging two **already sorted** lists. Let's start there. I am actually going to use generators here - makes the code a bit simpler - we can let the caller of `merge()` make a list if needed.

(I've intentionally made a mistake in the code).

In [17]:
def merge(l1, l2):
    # assumptions is that both l1 and l2 are sorted
    l1_index = 0
    l2_index = 0

    while l1_index < len(l1) and l2_index < len(l2):
        l1_value = l1[l1_index]
        l2_value = l2[l2_index]
        if l1_value < l2_value:
            l1_index = l1_index + 1
            yield l1_value
        else:
            l2_index = l2_index + 1
            yield l2_value
    # At this point one (or both) of the lists are "empty". If one is non-empty (and 
    # only one can be non-empty), just yield those values (they're already sorted)
    if l1_index < len(l1):
        yield from l1[l1_index+1:]
    if l2_index < len(l2):
        yield from l2[l2_index+1:]

Let's test our merge algorithm with two sorted lists:

In [18]:
list(merge([1, 4, 6], [2, 3, 8, 9]))

[1, 2, 3, 4, 6, 9]

Well, that's obviously not working, what happened to `8`?

Normally, I'd either move this to something like PyCharm and use the debugger, or just add print statements - but let's use `ic`:

In [19]:
def merge(l1, l2):
    # assumptions is that both l1 and l2 are sorted
    l1_index = 0
    l2_index = 0

    while l1_index < len(l1) and l2_index < len(l2):
        l1_value = ic(l1[l1_index])
        l2_value = ic(l2[l2_index])
        if l1_value < l2_value:
            l1_index = ic(l1_index + 1)
            yield ic(l1_value)
        else:
            l2_index = ic(l2_index + 1)
            yield ic(l2_value)
    # At this point one (or both) of the lists are "empty". If one is non-empty (and 
    # only one can be non-empty), just yield those values (they're already sorted)
    ic("At least one list is now empty")
    if l1_index < len(l1):
        ic("l1 non-empty")
        yield from ic(l1[l1_index+1:])
    if l2_index < len(l2):
        ic("l2 non-empty")
        yield from ic(l2[l2_index+1:])

In [20]:
list(merge([1, 4, 6], [2, 3, 8, 9]))

ic| l1[l1_index]: 1
ic| l2[l2_index]: 2
ic| l1_index + 1: 1
ic| l1_value: 1
ic| l1[l1_index]: 4
ic| l2[l2_index]: 2
ic| l2_index + 1: 1
ic| l2_value: 2
ic| l1[l1_index]: 4
ic| l2[l2_index]: 3
ic| l2_index + 1: 2
ic| l2_value: 3
ic| l1[l1_index]: 4
ic| l2[l2_index]: 8
ic| l1_index + 1: 2
ic| l1_value: 4
ic| l1[l1_index]: 6
ic| l2[l2_index]: 8
ic| l1_index + 1: 3
ic| l1_value: 6
ic| 'At least one list is now empty'
ic| 'l2 non-empty'
ic| l2[l2_index+1:]: [9]


[1, 2, 3, 4, 6, 9]

As we can see, most of the algorithm seems to be working - we are pulling off values `1`, then `2`, `3`, `4` and `6`.

Then we see that `l2` is non-empty, and it "contains" just a single element `9` - that's wrong, we know that it shoudl also contain `8` (it was never yielded before), so our issue is in the code there.

Easy fix, we just have the indexing wrong:

In [21]:
def merge(l1, l2):
    # assumptions is that both l1 and l2 are sorted
    l1_index = 0
    l2_index = 0

    while l1_index < len(l1) and l2_index < len(l2):
        l1_value = ic(l1[l1_index])
        l2_value = ic(l2[l2_index])
        if l1_value < l2_value:
            l1_index = ic(l1_index + 1)
            yield ic(l1_value)
        else:
            l2_index = ic(l2_index + 1)
            yield ic(l2_value)
    # At this point one (or both) of the lists are "empty". If one is non-empty (and 
    # only one can be non-empty), just yield those values (they're already sorted)
    ic("At least one list is now empty")
    if l1_index < len(l1):
        ic("l1 non-empty")
        yield from ic(l1[l1_index:])
    if l2_index < len(l2):
        ic("l2 non-empty")
        yield from ic(l2[l2_index:])

In [22]:
list(merge([1, 4, 6], [2, 3, 8, 9]))

ic| l1[l1_index]: 1
ic| l2[l2_index]: 2
ic| l1_index + 1: 1
ic| l1_value: 1
ic| l1[l1_index]: 4
ic| l2[l2_index]: 2
ic| l2_index + 1: 1
ic| l2_value: 2
ic| l1[l1_index]: 4
ic| l2[l2_index]: 3
ic| l2_index + 1: 2
ic| l2_value: 3
ic| l1[l1_index]: 4
ic| l2[l2_index]: 8
ic| l1_index + 1: 2
ic| l1_value: 4
ic| l1[l1_index]: 6
ic| l2[l2_index]: 8
ic| l1_index + 1: 3
ic| l1_value: 6
ic| 'At least one list is now empty'
ic| 'l2 non-empty'
ic| l2[l2_index:]: [8, 9]


[1, 2, 3, 4, 6, 8, 9]

Now everything seems correct.

Let's go ahead and now write the sort function itself. This is the recursive function that will keep splitting the unsorted lists, until they reach a single element, then merge the results as the recursion unwinds. (Note that the approach I use here could be improved substantially by not having to actually split lists into sub lists, but rather maintain pointers to start/stop locations - but that's more complex, so I'm taking the "easy" way out, for clarity)

In [23]:
def sort(data):
    if len(data) > 1:
        left = data[:len(data)//2]
        right = data[len(data)//2+1:]
    else:
        return data

Now, I'm not sure about whether the way I am splitting the array is correct, so let's use `ic` to view what's happening:

In [24]:
def sort(data):
    if len(data) > 1:
        left = ic(data[:len(data)//2])
        right = ic(data[len(data)//2+1:])
    else:
        return data

In [25]:
sort([5, 4, 3, 2, 1])

ic| data[:len(data)//2]: [5, 4]
ic| data[len(data)//2+1:]: [2, 1]


Ah, as you can see there's a mistake, let's fix that:

In [26]:
def sort(data):
    if len(data) > 1:
        left = ic(data[:len(data)//2])
        right = ic(data[len(data)//2:])
    else:
        return data

In [27]:
sort([1, 2, 3, 4, 5, 6])

ic| data[:len(data)//2]: [1, 2, 3]
ic| data[len(data)//2:]: [4, 5, 6]


In [28]:
sort([1, 2, 3])

ic| data[:len(data)//2]: [1]
ic| data[len(data)//2:]: [2, 3]


In [29]:
sort([1])

[1]

Ok, seems to be working, so let's continue implementing the sort:

In [30]:
def sort(data):
    if len(data) > 1:
        left = ic(data[:len(data)//2])
        right = ic(data[len(data)//2:])
        left_sorted = ic(sort(left))
        right_sorted = ic(sort(right))
        return list(merge(left_sorted, right_sorted))
    else:
        return data

In [31]:
sort([1])

[1]

In [32]:
sort([2, 1])

ic| data[:len(data)//2]: [2]
ic| data[len(data)//2:]: [1]
ic| sort(left): [2]
ic| sort(right): [1]
ic| l1[l1_index]: 2
ic| l2[l2_index]: 1
ic| l2_index + 1: 1
ic| l2_value: 1
ic| 'At least one list is now empty'
ic| 'l1 non-empty'
ic| l1[l1_index:]: [2]


[1, 2]

In [33]:
sort([5, 4, 6, 2, 3, 1, 8, 7, 9])

ic| data[:len(data)//2]: [5, 4, 6, 2]
ic| data[len(data)//2:]: [3, 1, 8, 7, 9]
ic| data[:len(data)//2]: [5, 4]
ic| data[len(data)//2:]: [6, 2]
ic| data[:len(data)//2]: [5]
ic| data[len(data)//2:]: [4]
ic| sort(left): [5]
ic| sort(right): [4]
ic| l1[l1_index]: 5
ic| l2[l2_index]: 4
ic| l2_index + 1: 1
ic| l2_value: 4
ic| 'At least one list is now empty'
ic| 'l1 non-empty'
ic| l1[l1_index:]: [5]
ic| sort(left): [4, 5]
ic| data[:len(data)//2]: [6]
ic| data[len(data)//2:]: [2]
ic| sort(left): [6]
ic| sort(right): [2]
ic| l1[l1_index]: 6
ic| l2[l2_index]: 2
ic| l2_index + 1: 1
ic| l2_value: 2
ic| 'At least one list is now empty'
ic| 'l1 non-empty'
ic| l1[l1_index:]: [6]
ic| sort(right): [2, 6]
ic| l1[l1_index]: 4
ic| l2[l2_index]: 2
ic| l2_index + 1: 1
ic| l2_value: 2
ic| l1[l1_index]: 4
ic| l2[l2_index]: 6
ic| l1_index + 1: 1
ic| l1_value: 4
ic| l1[l1_index]: 5
ic| l2[l2_index]: 6
ic| l1_index + 1: 2
ic| l1_value: 5
ic| 'At least one list is now empty'
ic| 'l2 non-empty'
ic| l2[l2_index:]:

[1, 2, 3, 4, 5, 6, 7, 8, 9]

Now, I can simply turn of all my "debugging" code, either by pulling the `ic` code out (which I probably would do once I have fully developed and tested my code, before putting into production), or, while I'm in the process of development turn it off (I can always turn it on in later parts of my code).

In [34]:
ic.disable()

In [35]:
sort([5, 4, 6, 2, 3, 1, 8, 7, 9])

[1, 2, 3, 4, 5, 6, 7, 8, 9]

You can also customize `ic`, and you'll find all of these in the library docs, but one I want to point out that can prove useful is configuring a `prefix`:

In [36]:
ic.enable()

In [37]:
ic(10 + 20)

ic| 10 + 20: 30


30

In [38]:
ic.configureOutput(prefix='jupyter| ')

In [39]:
ic(10+20)

jupyter| 10+20: 30


30

And the prefix can even be a callable, maybe to output the current datetime, or even just the perf_counter:

In [40]:
from time import perf_counter

In [41]:
ic.configureOutput(prefix=lambda: f"debug| {perf_counter():.4f}s| ")

In [42]:
sort([5, 4, 6, 2, 3, 1, 8, 7, 9])

debug| 2375948.2419s| data[:len(data)//2]: [5, 4, 6, 2]
debug| 2375948.2501s| data[len(data)//2:]: [3, 1, 8, 7, 9]
debug| 2375948.2573s| data[:len(data)//2]: [5, 4]
debug| 2375948.2639s| data[len(data)//2:]: [6, 2]
debug| 2375948.2703s| data[:len(data)//2]: [5]
debug| 2375948.2769s| data[len(data)//2:]: [4]
debug| 2375948.2829s| sort(left): [5]
debug| 2375948.2869s| sort(right): [4]
debug| 2375948.3312s| l1[l1_index]: 5
debug| 2375948.3356s| l2[l2_index]: 4
debug| 2375948.3398s| l2_index + 1: 1
debug| 2375948.3446s| l2_value: 4
debug| 2375948.3483s| 'At least one list is now empty'
debug| 2375948.3512s| 'l1 non-empty'
debug| 2375948.3543s| l1[l1_index:]: [5]
debug| 2375948.3581s| sort(left): [4, 5]
debug| 2375948.3623s| data[:len(data)//2]: [6]
debug| 2375948.3667s| data[len(data)//2:]: [2]
debug| 2375948.3718s| sort(left): [6]
debug| 2375948.3762s| sort(right): [2]
debug| 2375948.3800s| l1[l1_index]: 6
debug| 2375948.3838s| l2[l2_index]: 2
debug| 2375948.3874s| l2_index + 1: 1
debug| 

[1, 2, 3, 4, 5, 6, 7, 8, 9]

Since you can use any callable for the prefix, you could even use a closure - maybe you just want a counter that increments every time you output something with `ic`:

In [43]:
from datetime import datetime

def create_counter():
    count = -1

    def counter():
        nonlocal count
        count += 1
        return f"debug| {count} | {datetime.now().isoformat()} | "

    return counter

You can then use counter this way:

In [44]:
counter = create_counter()

In [45]:
counter()

'debug| 0 | 2024-06-22T14:19:23.718279 | '

In [46]:
counter()

'debug| 1 | 2024-06-22T14:19:23.720791 | '

In [47]:
counter()

'debug| 2 | 2024-06-22T14:19:23.723809 | '

And we can then use this counter with `ic` this way:

In [48]:
ic.configureOutput(prefix=create_counter())

In [49]:
sort([5, 3, 1, 2, 4])

debug| 0 | 2024-06-22T14:19:23.728496 | data[:len(data)//2]: [5, 3]
debug| 1 | 2024-06-22T14:19:23.736186 | data[len(data)//2:]: [1, 2, 4]
debug| 2 | 2024-06-22T14:19:23.744001 | data[:len(data)//2]: [5]
debug| 3 | 2024-06-22T14:19:23.790455 | data[len(data)//2:]: [3]
debug| 4 | 2024-06-22T14:19:23.797813 | sort(left): [5]
debug| 5 | 2024-06-22T14:19:23.803860 | sort(right): [3]
debug| 6 | 2024-06-22T14:19:23.809359 | l1[l1_index]: 5
debug| 7 | 2024-06-22T14:19:23.815403 | l2[l2_index]: 3
debug| 8 | 2024-06-22T14:19:23.821891 | l2_index + 1: 1
debug| 9 | 2024-06-22T14:19:23.827558 | l2_value: 3
debug| 10 | 2024-06-22T14:19:23.832918 | "At least one list is now empty": 'At least one list is now empty'
debug| 11 | 2024-06-22T14:19:23.838869 | 'l1 non-empty'
debug| 12 | 2024-06-22T14:19:23.844132 | l1[l1_index:]: [5]
debug| 13 | 2024-06-22T14:19:23.851353 | sort(left): [3, 5]
debug| 14 | 2024-06-22T14:19:23.858346 | data[:len(data)//2]: [1]
debug| 15 | 2024-06-22T14:19:23.917447 | data[le

[1, 2, 3, 4, 5]

Last customization I want to touch on is specifying custom representations.

Suppose we have this JSON:

In [50]:
json_data = '''{
    "a": [1, 2, 3],
    "b": {
        "b_1": 100,
        "b_2": {
            "b_2_1": 1000,
            "b_2_2": 2000
        }
    }
}'''

In [51]:
ic(json_data)

debug| 56 | 2024-06-22T14:19:24.342447 | json_data: ('{
                                                    '
                                                     '    "a": [1, 2, 3],
                                                    '
                                                     '    "b": {
                                                    '
                                                     '        "b_1": 100,
                                                    '
                                                     '        "b_2": {
                                                    '
                                                     '            "b_2_1": 1000,
                                                    '
                                                     '            "b_2_2": 2000
                                                    '
                                                     '        }
                                                    '
   

'{\n    "a": [1, 2, 3],\n    "b": {\n        "b_1": 100,\n        "b_2": {\n            "b_2_1": 1000,\n            "b_2_2": 2000\n        }\n    }\n}'

As you can see the output works, but is far from being very readable. 

If i were using a print statement, I would do something like this:

In [52]:
import json

repr_ = json.dumps(json.loads(json_data), indent=2)
print(repr_)

{
  "a": [
    1,
    2,
    3
  ],
  "b": {
    "b_1": 100,
    "b_2": {
      "b_2_1": 1000,
      "b_2_2": 2000
    }
  }
}


Now, let's basically hook this up to `ic`. Now, `ic` can handle all kinds of different objects, and they each need to get serialized differently. We do this by **registering** functions for a specific data type that `ic` will then be able to use (it uses the single dispatch functionality in Python that I cover in my Python 3: Deep Dive - Part 1 Udemy course).

One minor problem here is that we already have a function used to represent regular strings - although JSON is also just an ordinary string, we do want to register our custom display function only for these JSON strings, not for arbitrary strings - that would break the existing string representations in `ic`.

We could bypas the issue altogether, without involving `ic` by creating a new type, that basically just inherits from `str` and implement a custom representation:

In [53]:
class JSON(str):
    def __repr__(self):
        return json.dumps(json.loads(self), indent=2)

In [54]:
JSON(json_data)

{
  "a": [
    1,
    2,
    3
  ],
  "b": {
    "b_1": 100,
    "b_2": {
      "b_2_1": 1000,
      "b_2_2": 2000
    }
  }
}

And of course `ic` will simply use the new type's `__repr__`:

In [55]:
ic(JSON(json_data))

debug| 57 | 2024-06-22T14:19:24.409577 | JSON(json_data): {
                                                            "a": [
                                                              1,
                                                              2,
                                                              3
                                                            ],
                                                            "b": {
                                                              "b_1": 100,
                                                              "b_2": {
                                                                "b_2_1": 1000,
                                                                "b_2_2": 2000
                                                              }
                                                            }
                                                          }


{
  "a": [
    1,
    2,
    3
  ],
  "b": {
    "b_1": 100,
    "b_2": {
      "b_2_1": 1000,
      "b_2_2": 2000
    }
  }
}

But what if we want the actual representation to be one thing, and the output from `ic` to be another?

Easy enough, let's try it:

In [56]:
class JSON(str):
    def __repr__(self):
        return json.dumps(json.loads(self), indent=None, separators=(",", ":"))

In [57]:
JSON(json_data)

{"a":[1,2,3],"b":{"b_1":100,"b_2":{"b_2_1":1000,"b_2_2":2000}}}

In [58]:
from icecream import argumentToString

In [59]:
@argumentToString.register(JSON)
def JSON_pretty_print(json_str):
    return json.dumps(json.loads(json_str), indent=2)

And now, if we use `ic` to print the value of the object:

In [60]:
ic(JSON(json_data))

debug| 58 | 2024-06-22T14:19:24.444496 | JSON(json_data): {
                                                            "a": [
                                                              1,
                                                              2,
                                                              3
                                                            ],
                                                            "b": {
                                                              "b_1": 100,
                                                              "b_2": {
                                                                "b_2_1": 1000,
                                                                "b_2_2": 2000
                                                              }
                                                            }
                                                          }


{"a":[1,2,3],"b":{"b_1":100,"b_2":{"b_2_1":1000,"b_2_2":2000}}}

yet still retaining the default representation:

In [61]:
JSON(json_data)

{"a":[1,2,3],"b":{"b_1":100,"b_2":{"b_2_1":1000,"b_2_2":2000}}}

And there you go, a simple library that can be quite handy at times for quickly adding debugging "print" statements to your code.